In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Reading data

train = pd.read_csv('../input/ltfs-finhack-av-competition/train.csv')
test = pd.read_csv('../input/ltfs-finhack-av-competition/test.csv')

#Getting shapes of dataset
print("Train shape: ",train.shape)
print("Test shape: ",test.shape)



In [ ]:
unique_id = test['unique_id']

In [ ]:
train.head()


In [ ]:
test.head()

In [ ]:
train.describe()
test.describe()

In [ ]:
print(train.info())
print(test.info())

In [ ]:
#target variable
#Loan default is the failure to repay a loan according to the terms agreed to in the promissory note.

print(train['loan_default'].value_counts(normalize=True))
train['loan_default'].value_counts().plot.bar()

In [ ]:
#Univariate analysis - to see distribution of data and detect outliers if any
#Categorical features - Employement type

train['employment_type'].value_counts().plot.bar(figsize=(10,5),title='Employment')

In [ ]:
#Numerical features
train['disbursed_amt'].hist(bins=100)

Hypothesis-1 
Higher cns_score better score --> loan_default 0 (able to pay loan)

In [ ]:
train['cns_score'].describe()
train['cns_score'].value_counts()

In [ ]:
pd.crosstab(train['employment_type'],train['loan_default'],normalize=True)

In [ ]:
correlation = train.corr()
correlation['aadhar_flag'].sort_values(ascending=False) #voderid -0.8 correlation

In [ ]:
train_data = train.copy()
test_data = test.copy()

In [ ]:
train_data.drop(['voderid_flg'], axis=1, inplace=True)
test_data.drop(['voderid_flg'], axis=1, inplace=True)

In [ ]:
correlation.loc[['aadhar_flag','pan_flag','driving_flag','passport_flag','voderid_flg'],
                ['aadhar_flag','pan_flag','driving_flag','passport_flag','voderid_flg']]

In [ ]:
correlation['pri_disbursed_amt'].sort_values(ascending=False)#drop pri_sanctionedamt corr-0.998747

In [ ]:
train_data.drop(['pri_sanctioned_amt'],axis=1,inplace=True)
test_data.drop(['pri_sanctioned_amt'],axis=1,inplace=True)

In [ ]:
train_data.columns

In [ ]:
#Checking all flags
print(train_data['mobileno_avl_flag'].value_counts())  #varianc=0 drop variable
print(train_data['aadhar_flag'].value_counts()) 
print(train_data['pan_flag'].value_counts()) 
print(train_data['voderid_flg'].value_counts()) 
print(train_data['driving_flag'].value_counts()) 
print(train_data['passport_flag'].value_counts()) 
 

In [ ]:
train_data.drop(['mobileno_avl_flag'],axis=True,inplace=True)
test_data.drop(['mobileno_avl_flag'],axis=True,inplace=True)

In [ ]:
#checking for null values
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:

print(train['employment_type'].value_counts())

print(train['employment_type'].mode())


In [ ]:
#Filling nan values
train['employment_type'].fillna(train['employment_type'].mode()[0], inplace=True)
test['employment_type'].fillna(train['employment_type'].mode()[0], inplace=True)

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
train.columns

In [ ]:
# date of birth is an useless attribute 
#  the only thing we can extract the is the year of birth and from that 'age' feature can be generated.
# let's first convert the date into date-time format


train_data['date_of_birth'] = pd.to_datetime(train_data['date_of_birth'], errors='coerce')
train_data['disbursal_date'] = pd.to_datetime(train_data['disbursal_date'], errors='coerce')

test_data['date_of_birth'] = pd.to_datetime(test_data['date_of_birth'], errors='coerce')
test_data['disbursal_date'] = pd.to_datetime(test_data['disbursal_date'], errors='coerce')




In [ ]:
print(train_data.columns)
print(test_data.columns)

In [ ]:
#Feature engineering
# Age feature in years
train_data['Age'] = train_data['disbursal_date'].dt.year - train_data['date_of_birth'].dt.year
test_data['Age'] = test_data['disbursal_date'].dt.year - test_data['date_of_birth'].dt.year

#month from disbursal date can be of use. Income tax months - repay loan, festivals - vehicle on loan
train_data['disbursal_month'] = train_data['disbursal_date'].dt.month
test_data['disbursal_month'] = test_data['disbursal_date'].dt.month

In [ ]:
print(train_data['Age'].head(12))
print(test_data['Age'].head(12))
print(train_data['disbursal_month'].head(12))
print(test_data['disbursal_month'].head(12))


In [ ]:
train_data.drop(['date_of_birth','disbursal_date'], axis=1, inplace=True)
test_data.drop(['date_of_birth','disbursal_date'], axis=1, inplace=True)

In [ ]:
#Converting 'avg_account_age' and 'credit_history_length' into numerical features

train_data['credit_history_length_years'] = train_data['credit_history_length'].apply(lambda x: x.split(' ')[0])
train_data['credit_history_length_years'] = train_data['credit_history_length'].apply(lambda x: x.split('yrs')[0])

test_data['credit_history_length_years'] = test_data['credit_history_length'].apply(lambda x: x.split(' ')[0])
test_data['credit_history_length_years'] = test_data['credit_history_length'].apply(lambda x: x.split('yrs')[0])
print(train_data['credit_history_length_years'].head(10))
print(test_data['credit_history_length_years'].head(10))


In [ ]:
train_data['credit_history_length_months'] = train_data['credit_history_length'].apply(lambda x: x.split(' ')[1])
train_data['credit_history_length_months'] = train_data['credit_history_length_months'].apply(lambda x: x.split('mon')[0])

test_data['credit_history_length_months'] = test_data['credit_history_length'].apply(lambda x: x.split(' ')[1])
test_data['credit_history_length_months'] = test_data['credit_history_length_months'].apply(lambda x: x.split('mon')[0])
print(train_data['credit_history_length_months'].head())
print(test_data['credit_history_length_months'].head())
                                                                                            
                                                                                        

In [ ]:
train_data.info()

In [ ]:
#Converting dtype from object to float
train_data['credit_history_length_years']=train_data['credit_history_length_years'].astype('float')
train_data['credit_history_length_months']=train_data['credit_history_length_months'].astype('float')

test_data['credit_history_length_years']=test_data['credit_history_length_years'].astype('float')
test_data['credit_history_length_months']=test_data['credit_history_length_months'].astype('float')
#train_data['credit_history_length_years'] = train_data['credit_history_length_years'] * 12

In [ ]:
train_data['credit_history_length_months'].dtype

In [ ]:
test_data['credit_history_length_months'].head()

In [ ]:
train_data['credit_history_length_months2'] = train_data['credit_history_length_months'].apply(lambda x: x/12)
train_data['credit_history_length_months2'] = train_data['credit_history_length_months2'].round(decimals=2)


train_data['credit_history_length_years'] =train_data['credit_history_length_years']+ train_data['credit_history_length_months2']
train_data['credit_history_length_years'].head()


In [ ]:
test_data['credit_history_length_months2'] = test_data['credit_history_length_months'].apply(lambda x: x/12)
test_data['credit_history_length_months2'] = test_data['credit_history_length_months2'].round(decimals=2)

test_data['credit_history_length_years'] =test_data['credit_history_length_years']+ test_data['credit_history_length_months2']
test_data['credit_history_length_years'].head()

In [ ]:

train_data.drop(['credit_history_length','credit_history_length_months','credit_history_length_months2'], axis=1, inplace=True)
test_data.drop(['credit_history_length','credit_history_length_months','credit_history_length_months2'], axis=1, inplace=True)


In [ ]:
train_data.info()

In [ ]:
# Conversion of avg_account_age attribute to numerical feature
train_data['avg_account_age_years'] = train_data['avg_account_age'].apply(lambda x: x.split(' ')[0])
train_data['avg_account_age_years'] = train_data['avg_account_age_years'].apply(lambda x: x.split('yrs')[0])

print(train_data['avg_account_age_years'].head())
print(train_data['avg_account_age'].head())

test_data['avg_account_age_years'] = test_data['avg_account_age'].apply(lambda x: x.split(' ')[0])
test_data['avg_account_age_years'] = test_data['avg_account_age_years'].apply(lambda x: x.split('yrs')[0])


print('-----------------------------------')
print(test_data['avg_account_age_years'].head())
print(test_data['avg_account_age'].head())

In [ ]:
train_data['avg_account_age_months'] = train_data['avg_account_age'].apply(lambda x: x.split(' ')[1])
train_data['avg_account_age_months'] = train_data['avg_account_age_months'].apply(lambda x: x.split('mon')[0])

print(train_data['avg_account_age_months'].head())
print(train_data['avg_account_age'].head())

test_data['avg_account_age_months'] = test_data['avg_account_age'].apply(lambda x: x.split(' ')[1])
test_data['avg_account_age_months'] = test_data['avg_account_age_months'].apply(lambda x: x.split('mon')[0])


print('-----------------------------------')
print(test_data['avg_account_age_months'].head())
print(test_data['avg_account_age'].head())

In [ ]:
#Converting dtype from object to float
train_data['avg_account_age_years']=train_data['avg_account_age_years'].astype('float')
train_data['avg_account_age_months']=train_data['avg_account_age_months'].astype('float')

test_data['avg_account_age_years']=test_data['avg_account_age_years'].astype('float')
test_data['avg_account_age_months']=test_data['avg_account_age_months'].astype('float')

In [ ]:
train_data['avg_account_age_months'].dtype

In [ ]:
# Conversion from months to accumulative years only
train_data['avg_account_age_months2'] = train_data['avg_account_age_months'].apply(lambda x: x/12)
train_data['avg_account_age_months2'] = train_data['avg_account_age_months2'].round(decimals=2)


train_data['avg_account_age_years'] =train_data['avg_account_age_years']+ train_data['avg_account_age_months2']
print(train_data['avg_account_age_years'].head())
print(train_data['avg_account_age'].head())

In [ ]:
test_data['avg_account_age_months2'] = test_data['avg_account_age_months'].apply(lambda x: x/12)
test_data['avg_account_age_months2'] = test_data['avg_account_age_months2'].round(decimals=2)


test_data['avg_account_age_years'] =test_data['avg_account_age_years']+ test_data['avg_account_age_months2']
print(test_data['avg_account_age_years'].head())
print(test_data['avg_account_age'].head())

In [ ]:
train_data.drop(['avg_account_age','avg_account_age_months','avg_account_age_months2'],axis=1, inplace=True)
test_data.drop(['avg_account_age','avg_account_age_months','avg_account_age_months2'], axis=1, inplace=True)

In [ ]:
print(train_data['cns_score_description'].unique())
print(train_data['cns_score_description'].nunique())

In [ ]:

# encodings for bureau score(perform cns score distribution)

train_data['cns_score_description'] = train_data['cns_score_description'].replace('No Bureau History Available', 0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: Sufficient History Not Available', 0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: Not Enough Info available on the customer', 0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: No Activity seen on the customer (Inactive)',0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: No Updates available in last 36 months', 0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: Only a Guarantor', 0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('Not Scored: More than 50 active Accounts found',0)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('M-Very High Risk', 1)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('L-Very High Risk', 1)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('K-High Risk', 2)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('J-High Risk', 2)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('I-Medium Risk', 3)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('H-Medium Risk', 3)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('G-Low Risk', 4)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('F-Low Risk', 4)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('E-Low Risk', 4)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('D-Very Low Risk', 5)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('C-Very Low Risk', 5)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('B-Very Low Risk', 5)
train_data['cns_score_description'] = train_data['cns_score_description'].replace('A-Very Low Risk', 5)

# checing the values in bureau score
train_data['cns_score_description'].value_counts()

In [ ]:
#Encoding cns_score_description in test_data
print(test_data['cns_score_description'].nunique())
print(test_data['cns_score_description'].unique())

In [ ]:
test_data['cns_score_description'] = test_data['cns_score_description'].replace('No Bureau History Available', 0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('Not Scored: Sufficient History Not Available', 0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('Not Scored: Not Enough Info available on the customer', 0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('Not Scored: No Activity seen on the customer (Inactive)',0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('Not Scored: No Updates available in last 36 months', 0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('Not Scored: Only a Guarantor', 0)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('M-Very High Risk', 1)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('L-Very High Risk', 1)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('K-High Risk', 2)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('J-High Risk', 2)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('I-Medium Risk', 3)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('H-Medium Risk', 3)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('G-Low Risk', 4)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('F-Low Risk', 4)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('E-Low Risk', 4)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('D-Very Low Risk', 5)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('C-Very Low Risk', 5)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('B-Very Low Risk', 5)
test_data['cns_score_description'] = test_data['cns_score_description'].replace('A-Very Low Risk', 5)

# checing the values in bureau score
test_data['cns_score_description'].value_counts()

In [ ]:
#Drop unnecesarry attributes
train_data.drop(['unique_id','branch_id','supplier_id','manufacturer_id','current_pincode_id','state_id'], axis=1, inplace=True)
test_data.drop(['unique_id','branch_id','supplier_id','manufacturer_id','current_pincode_id','state_id'], axis=1, inplace=True)

In [ ]:
# One -hot encodng for employement type feature
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

print(train_data.columns)
print(test_data.columns)

In [ ]:
label = train['loan_default']
train_data.drop(['loan_default'], axis=1, inplace=True)

In [ ]:
test_data.drop(['loan_default'], axis=1, inplace=True)

In [ ]:
# applying standardization

# standardization

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
train_data = sc.fit_transform(train_data)
test_data = sc.transform(test_data)


In [ ]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='saga')
param_grid = {'C': [0.003,0.005,0.008,0.01,0.1,1], 'penalty': ['l1', 'l2']}
gridsearch = GridSearchCV(clf, param_grid,cv=5,scoring='roc_auc',refit=True) 
#cross_val=5 , metric roc auc score,refit will refit model on whole dataset
gridsearch.fit(train_data, label)
print(gridsearch.best_params_)'''



In [ ]:
''' final val score:  0.6236535202253027  C-0.005
    final val score:  0.6255618571430888  C-0.009
    final val score:  0.6249658474263933  C-0.01
    final val score:  0.6255209173762393  C-0.03
    final val score:  0.6256649213065772  C-0.1
    final val score:  0.6257040067515394  C-1.0
    final val score:  0.625706716037291   C-10
    final val score:  0.6257072031367716  C-100'''   


In [ ]:
# # # Models to Evaluate

# We will compare five different machine learning Cassification models:

# 1 - Logistic Regression
# 2 - Suport Vector Machine
# 3 - Random Forest Classification
# 4 = XGboost

# Function to calculate mean absolute error
def cross_val(model):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import roc_auc_score
    score = 0.0

    # stratified k-fold technique
    i=1
    skf =  StratifiedKFold(n_splits=5,random_state=0)

    for train_index, val_index in skf.split(train_data,label):
        print('\n{} of kfold {}'.format(i,skf.n_splits))
        xtr,xval = train_data[train_index],train_data[val_index]
        ytr,yval = label.loc[train_index],label.loc[val_index]

        model.fit(xtr,ytr)
        y_predicted = model.predict_proba(xval)
        score = score + roc_auc_score(yval, y_predicted[:,1]) #Passing only prob of positive class

        print('roc auc score: ',roc_auc_score(yval, y_predicted[:,1]))
        i = i + 1

    print('final val score: ',score/5)
    

# Takes in a model, trains the model, and evaluates the model on the test set
def predict_for_test(model):
    
   
    # Let's predict for test set
    test_predicted = model.predict_proba(test_data)
    y_pred = test_predicted[:,1]
    
    
    # Return the performance metric
    return y_pred

In [ ]:
'''from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='saga')
val_score = cross_val(lr)
print('LR: ',val_score)'''




In [ ]:
# # Random Forest Classification
'''
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10)
cross_val(rf)
'''

In [ ]:
# # Gradiente Boosting Classification
from xgboost import XGBClassifier
gb = XGBClassifier()
cross_val(gb)

In [ ]:
y_pred = predict_for_test(gb)

In [ ]:
#  let's create a submission file

#Create a  DataFrame with the passengers ids and our prediction regarding whether they survived or not

submission = pd.DataFrame({'UniqueID': unique_id,'loan_default': y_pred})

#Visualize the first 5 rows
submission.head()




In [ ]:
#Convert DataFrame to a csv file that can be uploaded
submission.to_csv('submission5_xgboost.csv',index=False)

Approach in this kernel -
drop certain features who had more correlation - voderid_flag, pri_sanctioned_amt
drop mobileavl_flag - 0 variance
new feature - disbursal_month

tried random forest but not giving good performance. 
Model used - xgboost.